In [5]:
from IPython.display import Latex
import numpy as np
from scipy.sparse import coo_matrix
import random
import time
from sklearn.cluster import KMeans
from scipy.spatial import distance
import pandas as pd
import os
from PIL import Image
import copy
import matplotlib.pyplot as plt
import statistics
import numpy as np
from scipy.sparse import coo_matrix
import random
from numpy.linalg import inv
from sklearn.cluster import KMeans
import pandas as pd
import math
import scipy.sparse.linalg as ll
import matplotlib as mpl
import numpy.linalg as linalg
import numpy as np
import numpy.matlib
from scipy.stats import multivariate_normal as mvn
import matplotlib.pyplot as plt
from sklearn import preprocessing
from scipy.stats import norm
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import OneClassSVM
from sklearn import metrics

In [ ]:
df = pd.read_pickle('stocksandindicators.pickle')
df_AAPL= df['AAPL']
historical_df = df_AAPL
for i in range(1,8): # for 7 days
    historical_df["Open_b_"+str(i)] = df_AAPL['Open'].shift(i)
    historical_df["High_b_"+str(i)] = df_AAPL['High'].shift(i)
    historical_df["Low_b_"+str(i)]  = df_AAPL['Low'].shift(i)
    historical_df["Close_b_"+str(i)]= df_AAPL['Close'].shift(i)
    historical_df['Volume_b_'+str(i)]      = df_AAPL['Volume'].shift(i)
    historical_df["volume_fi_b_"+str(i)] = df_AAPL['volume_fi'].shift(i)

historical_df = historical_df.dropna() 
print("Historical Data Shape: ", historical_df.shape)
historical_df.tail(3)

In [6]:
df = pd.read_pickle('stocksandindicators.pickle')
df_AAPL = pd.DataFrame()
df_AAPL= df['AAPL']
df_AAPL['Volume'].fillna(value=0, inplace=True)
df_AAPL['volume_fi'].fillna(value=0, inplace=True)

df_AAPL['Open'].fillna(method='ffill', inplace=True)
df_AAPL['High'].fillna(method='ffill', inplace=True)
df_AAPL['Low'].fillna(method='ffill', inplace=True)
df_AAPL['Close'].fillna(method='ffill', inplace=True)
df_AAPL['Close_lag'] = df_AAPL['Close'].shift(0)
df_AAPL.tail(3)
df_AAPL.set_index('Dates', inplace=True)
X = df_AAPL.loc[:, (df_AAPL.columns != 'Close') & (df_AAPL.columns != 'Close_lag')]
y = df_AAPL['Close_lag']
features = np.array(X.columns)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.05, random_state=642,shuffle=False)
rf = RandomForestRegressor(n_estimators = 1000, random_state = 5)
rf.fit(X_train, np.ravel(y_train));
predictions = rf.predict(X_test)
df_Result = pd.DataFrame(y_test)
df_Result['Predicted'] = predictions

In [ ]:
plt.figure(figsize=(25,15), facecolor='w', edgecolor='k')
ax = plt.gca()  
plt.plot(df_Result['Close_lag'], color = 'red', label = 'Real Closing Price')
plt.plot(df_Result['Predicted'], color = 'blue', label = 'Predicted Closing Price')
plt.title('Price Prediction', fontsize=40)
df_test = df_Result.reset_index()
x = df_test.index
for tick in ax.xaxis.get_major_ticks():
    tick.label1.set_fontsize(18)
for tick in ax.yaxis.get_major_ticks():
    tick.label1.set_fontsize(18)
plt.xlabel('Time', fontsize=40)
plt.ylabel('Price(USD) [Closed]', fontsize=40)
plt.legend(loc=2, prop={'size': 25})
plt.show()

In [ ]:
importances = rf.feature_importances_
indices = np.argsort(importances)[::-1]
feat_labels = X.columns
plt.figure(figsize = (20,20))
plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), [feat_labels[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()

In [ ]:
def RF_mod(X,y,lenf, dates):
    train_size = int(len(X)*0.95)
    test_size= len(X)- train_size
    x_train, x_test = X[0:train_size], X[train_size:len(X)]
    y_train, y_test = y[0:train_size], y[train_size:len(y)]
    datestrain, datestest = dates[0:train_size], dates[train_size:len(dates)]

    scalerX = StandardScaler().fit(x_train)
    scalerY = StandardScaler().fit(y_train)
    
    x_train = scalerX.transform(x_train)
    y_train = scalerY.transform(y_train)
    x_test = scalerX.transform(x_test)
    y_test = scalerY.transform(y_test)
      
    #x_train = x_train.reshape((x_train.shape[0],lenf,1))
    #x_test = x_test.reshape((x_test.shape[0],lenf,1))
    #earlystopping = callbacks.EarlyStopping(monitor ="val_loss",  
                                        #mode ="min", patience = 5,  
                                        #restore_best_weights = True) 
    
    
    #x_train, x_test, y_train, y_test = train_test_split(X,y,test_size=0.05, random_state=642,shuffle=False)
    model = RandomForestRegressor(n_estimators = 1000, random_state = 5)
    

    model.fit(x_train, y_train)
    trainPredict = model.predict(x_train)
    testPredict = model.predict(x_test)

    # invert predictions
    trainPredict_y = scalerY.inverse_transform(trainPredict)
    testPredict_y = scalerY.inverse_transform(testPredict)
    y_true_train = y[0:train_size]
    y_true_test = y[-test_size:]

    df_ytrue_train = pd.DataFrame({'Dates' : datestrain.values,'Close': y_true_train.ravel(), 'Close Predicted' : trainPredict_y.flatten()})
    df_ytrue_test = pd.DataFrame({'Dates' : datestest.values, 'Close': y_true_test.ravel(), 'Close Predicted' : testPredict_y.flatten()})
    #print('Model accuracy (%)')
    Y_t=scalerY.inverse_transform(y_train)
    y_test_T = scalerY.inverse_transform(y_test)
    Accuracy = ((1-(metrics.mean_absolute_error(Y_t, trainPredict_y)/Y_t.mean()))*100)
    #print('')
    #print('Prediction performance')
    MAE =((metrics.mean_absolute_error(y_test_T, testPredict)/y_test_T.mean())*100)
    MSE=metrics.mean_squared_error(y_test_T,  testPredict)
    RMSE=np.sqrt(MSE)
    R2= metrics.r2_score(y_test_T,  testPredict)
    return df_ytrue_train, df_ytrue_test, Accuracy, MAE, MSE, RMSE, R2, datestrain, datestest
def preprocess(df):
    df.drop(columns=['Split Coef'], inplace=True)
    df.set_index('Dates', inplace=True)
    X = df.loc[:, collist]
    y = df[['Close']].values
    features = np.array(X.columns)
    return X, y, features    
def maketrainplots(df_ytrue_train, indic_type, key,df_type):
    fig, ax = plt.subplots(figsize= (9,5))
    plt.plot(df_ytrue_train['Dates'], df_ytrue_train['Close'],
                                label='Closing Price')
    plt.plot(df_ytrue_train['Dates'], df_ytrue_train['Close Predicted'],'r--',
                                label='Predicted Closing Price')
    plt.legend()
    plt.savefig(f'data_project/Price Plots/{indic_type}/train_{indic_type}_{df_type}_{key}.png')
    plt.close()
def maketestplots(df_ytrue_test, indic_type, key, df_type):
    fig, ax = plt.subplots(figsize= (9,5))
    plt.plot(df_ytrue_test['Dates'], df_ytrue_test['Close'],
                                label='Closing Price')
    plt.plot(df_ytrue_test['Dates'], df_ytrue_test['Close Predicted'],'r--',
                                label='Predicted Closing Price')
    plt.legend()
    plt.savefig(f'data_project/Price Plots/{indic_type}/test_{indic_type}_{df_type}_{key}.png')
    plt.close()
if __name__ == "__main__":   
    obj = pd.read_pickle(r'data_project/stocksandindicators.pickle')
    sp500_df = pd.read_csv('data_project/Stocks in the SP 500 Index.csv')
    indic_type = 'Trend'
    df_indicators = pd.read_csv('data_project/LassoCoefficientsAllModels_partial_trend.csv')
    df_indicators.rename(columns={'Unnamed: 0' : 'Stock'}, inplace=True)
    df_indicators.set_index('Stock', inplace=True)
    #sp500_df['Dividend yield'] = sp500_df['Dividend yield'].map(lambda x: x.rstrip('%'))
    #sp500_df['Dividend yield'] = sp500_df['Dividend yield'].apply(pd.to_numeric)
    #df_dividend = sp500_df.loc[sp500_df['Dividend yield']>0]
    #df_not_dividend = sp500_df.loc[sp500_df['Dividend yield']==0]
    
    df_sector_IT = sp500_df.loc[sp500_df['GICS Sector']=='Information Technology']
    df_sector_CS = sp500_df.loc[sp500_df['GICS Sector']=='Communication Services']
    df_sector_CD = sp500_df.loc[sp500_df['GICS Sector']=='Consumer Discretionary']
    df_sector_Fin = sp500_df.loc[sp500_df['GICS Sector']=='Financials']
    df_sector_HC = sp500_df.loc[sp500_df['GICS Sector']=='Health Care']
    df_sector_Energy = sp500_df.loc[sp500_df['GICS Sector']=='Energy']
    df_sector_Staples = sp500_df.loc[sp500_df['GICS Sector']=='Consumer Staples']
    df_sector_Utils = sp500_df.loc[sp500_df['GICS Sector']=='Utilities']
    df_sector_Materials = sp500_df.loc[sp500_df['GICS Sector']=='Materials']
    df_sector_Industrials = sp500_df.loc[sp500_df['GICS Sector']=='Industrials']
    df_sector_RE = sp500_df.loc[sp500_df['GICS Sector']=='Real Estate']
    
    #df_dividend = df_dividend['Symbol'].tolist()
    #df_not_dividend = df_not_dividend['Symbol'].tolist()
    
    df_sector_IT = df_sector_IT['Symbol'].tolist()
    df_sector_CS = df_sector_CS['Symbol'].tolist()
    df_sector_CD = df_sector_CD['Symbol'].tolist()
    df_sector_Fin = df_sector_Fin['Symbol'].tolist()
    df_sector_HC = df_sector_HC['Symbol'].tolist()
    df_sector_Energy = df_sector_Energy['Symbol'].tolist()
    df_sector_Staples = df_sector_Staples['Symbol'].tolist()
    df_sector_Utils = df_sector_Utils['Symbol'].tolist()
    df_sector_Materials = df_sector_Materials['Symbol'].tolist()
    df_sector_Industrials = df_sector_Industrials['Symbol'].tolist()
    df_sector_RE = df_sector_RE['Symbol'].tolist()
    
    #fulllist = [df_dividend,df_not_dividend,df_sector_IT,df_sector_CS,df_sector_CD,df_sector_Fin,df_sector_HC,
    #            df_sector_Energy,df_sector_Staples,df_sector_Utils,df_sector_Materials,df_sector_Industrials,df_sector_RE]
   # fulllist = [df_sector_Staples,df_sector_Utils,df_sector_Materials,df_sector_Industrials,df_sector_RE]#
    fulllist = [df_sector_Energy,df_sector_IT,df_sector_CS,df_sector_CD,df_sector_Fin,df_sector_HC,
                df_sector_Staples,df_sector_Utils,df_sector_Materials,df_sector_Industrials,df_sector_RE]
    #splitnames = ['dividend', 'no dividend','Information Technology','Communication Services','Consumer Discretionary','Financials','Health Care',
    #              'Energy','Consumer Staples','Utilities','Materials','Industrials','Real Estate']
    #splitnames = ['Consumer Staples','Utilities','Materials','Industrials','Real Estate']#
    splitnames = ['Energy','Information Technology','Communication Services','Consumer Discretionary','Financials','Health Care',
                 'Consumer Staples','Utilities','Materials','Industrials','Real Estate']
    ct = 0
    
    for dfl in fulllist:
        df_temp = df_indicators[df_indicators.index.isin(dfl)]
        lendft = len(df_temp)
        params_list=[]
        for col in df_temp:
            count0 = df_temp[col].isin([0]).sum()
            if (lendft - count0)/lendft < 0.7:
                df_temp.drop(col, axis=1, inplace=True)
            else:
                params_list.append(f'{col}: {count0}')
        
        collist = df_temp.columns
        df_errors = pd.DataFrame()
        df_type = splitnames[ct]
        with open(f'data_project/Price Plots/{indic_type}/parameters_{indic_type}_{df_type}.txt', 'w') as fp:
            for item in params_list:
                # write each item on a new line
                fp.write("%s\n" % item)
        for key in obj:
            
            if key in df_temp.index:
                #idx_list.append(key)
                X, y, features = preprocess(obj[key])
                df_ytrue_train, df_ytrue_test, Accuracy, MAE, MSE, RMSE, R2, datestrain, datestest = RF_mod(X,y,len(features), obj[key].index)
                maketrainplots(df_ytrue_train, indic_type, key, df_type)
                maketestplots(df_ytrue_test, indic_type, key, df_type)
                #print(key, X, features)
                df_temp_errors = pd.DataFrame(columns=['Accuracy','MAE','MSE','RMSE','R2', 'Start Date Train', 'Start Date Test', 'End Date Test'])
                df_temp_errors.loc[0] = [Accuracy, MAE, MSE,RMSE,R2, datestrain[0], datestest[0], datestest[-1]]
                df_errors =pd.concat([df_errors,df_temp_errors])
        df_errors.index=dfl
        df_errors.to_csv(f'data_project/Price Plots/{indic_type}/errors_{indic_type}_{df_type}.csv')
        print(splitnames[ct])
        ct+=1
    